# Plot the MCMC results

In [ ]:
import h5py
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [ ]:
PARAMETER_NAMES = [
    r'$B_G\;[\mathrm{GeV}^{-2}]$',
    r'$\alpha_{\rm shadowing}$',
    r'$y_{{\rm loss},2}$',
    r'$y_{{\rm loss},4}$',
    r'$y_{{\rm loss},6}$',
    r'$\sigma_{y_{\rm loss}}$',
    r'$\alpha_{\rm rem}$',
    r'$\lambda_B$',
    r'$\sigma_x^{\rm string}\;[{\rm fm}]$',
    r'$\sigma_\eta^{\rm string}$',
    r'$\alpha_{{\rm string}\;{\rm tilt}}$',
    r'$\alpha_{\rm preFlow}$',
    r'$\eta_0$',
    r'$\eta_2$',
    r'$\eta_4$',
    r'$\zeta_{\rm max}$',
    r'$T_{\zeta,0}\;[{\rm GeV}]$',
    r'$\sigma_{\zeta,+}\;[{\rm GeV}]$',
    r'$\sigma_{\zeta,-}\;[{\rm GeV}]$',
    r'$e_{\rm sw}\;[{\rm GeV}/{\rm fm}^3]$'
    ]

prior_ranges = [
    (1,25),
    (0,1),
    (0,2),
    (1,3),
    (1,4),
    (0.1,0.8),
    (0,1),
    (0,1),
    (0.1,0.8),
    (0.1,1),
    (0,1),
    (0,2),
    (0.001,0.3),
    (0.001,0.3),
    (0.001,0.3),
    (0,0.2),
    (0.15,0.25),
    (0.01,0.15),
    (0.005,0.1),
    (0.15,0.5)
] 

def read_h5_file(PATH_h5file_chains):
    # Open the HDF5 file in read mode
    with h5py.File(PATH_h5file_chains, 'r') as hf:
        # Iterate over the keys in the file
        for key in hf.keys():
            # Access the dataset corresponding to the key
            dataset = hf[key]

            n_walkers = dataset.shape[0]
            n_steps = dataset.shape[1] # this is the originally set value divided by 10
            n_parameters = dataset.shape[2]

            # Create a 3D array to store the data
            data_array = np.zeros((n_walkers, n_steps, n_parameters))

            # Iterate through the dataset and store the data in the array
            for i in range(n_walkers):
                for j in range(n_steps):
                    data_array[i, j, :] = dataset[i, j, :]

    return n_walkers, n_steps, n_parameters, data_array

def read_real_parameters(file):
    with open(file, "rb") as pf:
        data = pickle.load(pf)
    return data['1000']['parameter']


In [ ]:
def plot_chain(n_walkers,n_steps,n_parameters,data_array,filename):
    num_columns = 3

    # Calculate the number of rows needed based on the number of parameters and columns
    num_rows = (n_parameters + num_columns - 1) // num_columns

    fig, axs = plt.subplots(num_rows, num_columns, figsize=(10, 7), sharex=False)
    axs = axs.flatten()

    for param_index in range(n_parameters):
        # Plot each walker's data for the parameter
        for walker_index in range(n_walkers):
            axs[param_index].plot(range(n_steps), data_array[walker_index, :, param_index], color='g', alpha=0.5)

        axs[param_index].set_ylabel(PARAMETER_NAMES[param_index])
        axs[param_index].set_xlim([0.,n_steps])

        axs[param_index].set_ylim([prior_ranges[param_index][0],prior_ranges[param_index][1]])

    # Specify indices for which to display x-axis labels
    indices_with_labels = [17, 18, 19]

    # Set x-axis labels and numbers for specified indices
    for i, ax in enumerate(axs):
        if i in indices_with_labels:
            ax.set_xlabel('Steps')
            ticks = axs[param_index].get_xticks()
            axs[param_index].set_xticks(ticks)
            axs[param_index].set_xticklabels([int(tick) for tick in ticks])
        else:
            ax.set_xticklabels([])

    # Hide the empty subplots
    for i in range(n_parameters, num_columns * num_rows):
        fig.delaxes(axs[i])

    plt.tight_layout()
    plt.savefig(f"{filename}.pdf")

In [ ]:
def plot_corner_1dataset(n_parameters,data_array,filename):
    samples = data_array[:,:,:].reshape((-1,n_parameters))

    # Create subplots
    fig, axes = plt.subplots(n_parameters, n_parameters, figsize=(25, 25), sharex='col', sharey='row')

    # Loop over each parameter pair
    for i in range(n_parameters):
        for j in range(n_parameters):
            ax = axes[i, j]

            if i == j:
                ax = fig.add_subplot(n_parameters, n_parameters, i * n_parameters + j + 1)
                ax.hist(samples[:, i], bins=20, color='g', histtype='step', density=True)
                ax.set_yticks([])
                ax.set_yticklabels([])
                if i != n_parameters-1:
                    ax.set_xticks([])
                    ax.set_xticklabels([])
                if i == n_parameters-1:
                    ax.tick_params(axis='x', rotation=45, labelsize=10)

                # Calculate percentiles
                percentiles = np.percentile(samples[:, i], [16, 50, 84])
                median = percentiles[1]
                upper = percentiles[2] - median
                lower = median - percentiles[0]

                ax.annotate(fr'${median:.2f}_{{-{lower:.2f}}}^{{+{upper:.2f}}}$', xy=(0.5, 1.05), xycoords='axes fraction', ha='center')

            # Only fill the lower triangle
            if i > j:
                ax.hist2d(samples[:, j], samples[:, i], bins=20, cmap='Greens')

            # Set labels on the lowest and leftmost plots
            if i == n_parameters - 1:
                ax.set_xlabel(PARAMETER_NAMES[j])
            if j == 0:
                ax.set_ylabel(PARAMETER_NAMES[i])

            if i < j:
                ax.axis('off')

    # Remove space between subplots
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    # Remove ticks and labels for the first and last plots
    axes[0, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    axes[n_parameters - 1, n_parameters - 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

    # Rotate x-axis tick labels
    for ax in axes[-1]:
        ax.tick_params(axis='x', rotation=45, labelsize=10)

    # Rotate y-axis tick labels
    for ax in axes[:, 0]:
        ax.tick_params(axis='y', rotation=45, labelsize=10)

    plt.savefig(f"{filename}.pdf")

In [ ]:
def plot_corner_compare_datasets(n_parameters,data_array1,data_array2,real_parameters,filename):
    samples1 = data_array1[:,:,:].reshape((-1,n_parameters))
    samples2 = data_array2[:,:,:].reshape((-1,n_parameters))

    # Create subplots
    fig, axes = plt.subplots(n_parameters, n_parameters, figsize=(25, 25), sharex='col', sharey='row')

    # Loop over each parameter pair
    for i in range(n_parameters):
        for j in range(n_parameters):
            ax = axes[i, j]

            if i == j:
                ax = fig.add_subplot(n_parameters, n_parameters, i * n_parameters + j + 1)
                ax.hist(samples1[:, i], bins=20, color='g', histtype='step', density=True)
                ax.hist(samples2[:, i], bins=20, color='magenta', histtype='step', density=True)
                ax.set_yticks([])
                ax.set_yticklabels([])
                if i != n_parameters-1:
                    ax.set_xticks([])
                    ax.set_xticklabels([])
                if i == n_parameters-1:
                    ax.tick_params(axis='x', rotation=45, labelsize=10)

                # Calculate percentiles
                percentiles1 = np.percentile(samples1[:, i], [16, 50, 84])
                median1 = percentiles1[1]
                upper1 = percentiles1[2] - median1
                lower1 = median1 - percentiles1[0]
                axes[0,j].annotate(fr'${median1:.2f}_{{-{lower1:.2f}}}^{{+{upper1:.2f}}}$', xy=(0.5, 1.05), xycoords='axes fraction', ha='center',color='g')

                percentiles2 = np.percentile(samples2[:, i], [16, 50, 84])
                median2 = percentiles2[1]
                upper2 = percentiles2[2] - median2
                lower2 = median2 - percentiles2[0]
                axes[0,j].annotate(fr'${median2:.2f}_{{-{lower2:.2f}}}^{{+{upper2:.2f}}}$', xy=(0.5, 1.25), xycoords='axes fraction', ha='center',color='magenta')

                # Add a vertical line representing the real value
                real_value = real_parameters[i]
                ax.axvline(x=real_value, color='k', linestyle='-')

            # Only fill the lower triangle
            if i > j:
                ax.hist2d(samples1[:, j], samples1[:, i], bins=20, cmap='Greens')
            if i < j:
                ax.hist2d(samples2[:, j], samples2[:, i], bins=20, cmap='RdPu')

            # Set labels on the lowest and leftmost plots
            if i == n_parameters - 1:
                ax.set_xlabel(PARAMETER_NAMES[j])
            if j == 0:
                if i == 0:
                    ax.set_ylabel(PARAMETER_NAMES[i],labelpad=25)
                else:
                    ax.set_ylabel(PARAMETER_NAMES[i])

    # Remove space between subplots
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)

    # Remove ticks and labels for the first and last plots
    #axes[0, 0].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)
    axes[n_parameters - 1, n_parameters - 1].tick_params(axis='both', which='both', bottom=False, left=False, labelbottom=False, labelleft=False)

    # Rotate x-axis tick labels
    for ax in axes[-1]:
        ax.tick_params(axis='x', rotation=45, labelsize=10)

    # Rotate y-axis tick labels
    for ax in axes[:, 0]:
        ax.tick_params(axis='y', rotation=45, labelsize=10)

    plt.savefig(f"{filename}.pdf")

In [ ]:
def parametrization_eta_over_s_vs_mu_B(eta_0, eta_2, eta_4, mu_B):
    eta_over_s = np.zeros_like(mu_B)  # Create an array to store the results

    # Compute eta_over_s for different ranges of mu_B
    eta_over_s[(mu_B >= 0) & (mu_B <= 0.2)] = eta_0 + (eta_2 - eta_0) * (mu_B[(mu_B >= 0) & (mu_B <= 0.2)] / 0.2)
    eta_over_s[(mu_B > 0.2) & (mu_B < 0.4)] = eta_2 + (eta_4 - eta_2) * ((mu_B[(mu_B > 0.2) & (mu_B < 0.4)] - 0.2) / 0.2)
    eta_over_s[mu_B >= 0.4] = eta_4

    return eta_over_s

def plot_eta_s_posterior(true_parameters,chain_parameters,filename):
    mu_B = np.linspace(0.,0.6,100)
    eta_s_parameters_true = true_parameters[12:15]
    true_eta_s = parametrization_eta_over_s_vs_mu_B(eta_s_parameters_true[0],eta_s_parameters_true[1],eta_s_parameters_true[2],mu_B)

    chain_array = chain_parameters[3]
    samples = chain_array[:,:,:].reshape((-1,chain_array.shape[2]))
    samples_eta_s = samples[:,12:15]

    posterior_eta_s = []
    for s in range(samples_eta_s.shape[0]):
        eta0 = samples_eta_s[s,0]
        eta2 = samples_eta_s[s,1]
        eta4 = samples_eta_s[s,2]
        posterior_eta_s.append(parametrization_eta_over_s_vs_mu_B(eta0,eta2,eta4,mu_B))
    posterior_eta_s = np.array(posterior_eta_s)
    median = np.percentile(posterior_eta_s, 50, axis=0)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,4),
                         sharex=False, sharey=False, constrained_layout=True)
    ax.plot(mu_B, true_eta_s, 'k--', label='Truth', lw=2)
    ax.plot(mu_B, median, 'g-', label="Marginalzied median", lw=2)

    for CL in [68,95,99.7]:
        low, high = np.percentile(posterior_eta_s, [50-CL/2., 50+CL/2.], axis=0)
        ax.fill_between(mu_B, low, high, alpha=0.2, color='g')
        ax.annotate("{}%CL".format(CL), xy=(mu_B[-1]*0.85, low[-1]), va="center", ha="left")

    ax.legend(loc='upper left')
    ax.set_xlabel('$\mu_B$ [GeV]')
    ax.set_ylabel('$\eta/s$')
    ax.set_xlim([0.,0.6])
    plt.savefig(f'{filename}.pdf')

In [ ]:
def parametrization_zeta_over_s_vs_T(zeta_max, T_zeta0, sigma_plus, sigma_minus, T, mu_B):
    T_zeta_muB = T_zeta0 - 0.15 * mu_B**2.

    # Compute zeta_over_s for different conditions using NumPy's vectorized operations
    zeta_over_s = np.where(T < T_zeta0,
                           zeta_max * np.exp(-(T - T_zeta_muB)**2. / (2. * sigma_minus**2.)),
                           zeta_max * np.exp(-(T - T_zeta_muB)**2. / (2. * sigma_plus**2.)))

    return zeta_over_s

def plot_zeta_s_posterior(true_parameters,chain_parameters,filename):
    T = np.linspace(0.,0.5,100)
    zeta_s_parameters_true = true_parameters[15:19]
    true_zeta_s = parametrization_zeta_over_s_vs_T(zeta_s_parameters_true[0],zeta_s_parameters_true[1],zeta_s_parameters_true[2],zeta_s_parameters_true[3],T,0.)

    chain_array = chain_parameters[3]
    samples = chain_array[:,:,:].reshape((-1,chain_array.shape[2]))
    samples_zeta_s = samples[:,15:19]

    posterior_zeta_s = []
    for s in range(samples_zeta_s.shape[0]):
        zeta_max = samples_zeta_s[s,0]
        T_zeta0 = samples_zeta_s[s,1]
        sigma_plus = samples_zeta_s[s,2]
        sigma_minus = samples_zeta_s[s,3]
        posterior_zeta_s.append(parametrization_zeta_over_s_vs_T(zeta_max,T_zeta0,sigma_plus,sigma_minus,T,0.))
    posterior_zeta_s = np.array(posterior_zeta_s)
    median = np.percentile(posterior_zeta_s, 50, axis=0)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,4),
                         sharex=False, sharey=False, constrained_layout=True)
    ax.plot(T, true_zeta_s, 'k--', label='Truth', lw=2)
    ax.plot(T, median, 'g-', label="Marginalzied median", lw=2)

    for CL in [68,95,99.7]:
        low, high = np.percentile(posterior_zeta_s, [50-CL/2., 50+CL/2.], axis=0)
        ax.fill_between(T, low, high, alpha=0.2, color='g')
        ax.annotate("{}%CL".format(CL), xy=(T[-1]*0.85, high[-1]), va="center", ha="left")

    ax.legend(loc='upper left')
    ax.set_xlabel('$T$ [GeV]')
    ax.set_ylabel('$\zeta/s$')
    ax.set_xlim([0.,0.5])
    plt.savefig(f'{filename}.pdf')

In [ ]:
def parametrization_y_loss_vs_y_init(yloss_2, yloss_4, yloss_6, y_init):
    # Create arrays to store the results
    y_loss = np.zeros_like(y_init)

    # Compute y_loss for different ranges of y_init using NumPy's vectorized operations
    y_loss[(y_init >= 0) & (y_init <= 2)] = yloss_2 * (y_init[(y_init >= 0) & (y_init <= 2)] / 2.)
    y_loss[(y_init > 2) & (y_init < 4)] = yloss_2 + (yloss_4 - yloss_2) * ((y_init[(y_init > 2) & (y_init < 4)] - 2.) / 2.)
    y_loss[y_init >= 4] = yloss_4 + (yloss_6 - yloss_4) * ((y_init[y_init >= 4] - 4.) / 2.)

    return y_loss

def plot_yloss_posterior(true_parameters,chain_parameters,filename):
    yinit = np.linspace(0.,6.2,100)
    yloss_parameters_true = true_parameters[2:5]
    true_yloss = parametrization_y_loss_vs_y_init(yloss_parameters_true[0],yloss_parameters_true[1],yloss_parameters_true[2],yinit)

    chain_array = chain_parameters[3]
    samples = chain_array[:,:,:].reshape((-1,chain_array.shape[2]))
    samples_yloss = samples[:,2:5]

    posterior_yloss = []
    for s in range(samples_yloss.shape[0]):
        yloss2 = samples_yloss[s,0]
        yloss4 = samples_yloss[s,1]
        yloss6 = samples_yloss[s,2]
        posterior_yloss.append(parametrization_y_loss_vs_y_init(yloss2,yloss4,yloss6,yinit))
    posterior_yloss = np.array(posterior_yloss)
    median = np.percentile(posterior_yloss, 50, axis=0)

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,4),
                         sharex=False, sharey=False, constrained_layout=True)
    ax.plot(yinit, true_yloss, 'k--', label='Truth', lw=2)
    ax.plot(yinit, median, 'g-', label="Marginalzied median", lw=2)

    for CL in [68,95,99.7]:
        low, high = np.percentile(posterior_yloss, [50-CL/2., 50+CL/2.], axis=0)
        ax.fill_between(yinit, low, high, alpha=0.2, color='g')
        ax.annotate("{}%CL".format(CL), xy=(yinit[-1]*0.85, high[-1]), va="center", ha="left")

    # Fill the area above the line y = x with gray
    ax.fill_between(yinit, yinit, 4.5, where=(yinit <= 4.5), color='gray', alpha=0.2)

    ax.legend(loc='upper left')
    ax.set_xlabel('$y_{\mathrm{init}}$')
    ax.set_ylabel('$\langle y_{\mathrm{loss}}\\rangle$')
    ax.set_xlim([0.,6.2])
    plt.savefig(f'{filename}.pdf')

In [ ]:
PATH_h5file_PCGP_noPCA = './mcmc/mcmc_PCGP_noPCA/chain.h5'
PATH_h5file_PCGP_PCA = './mcmc/mcmc_PCGP_PCA/chain.h5'

PATH_h5file_PCSK_noPCA = './mcmc/mcmc_PCSK_noPCA/chain.h5'
PATH_h5file_PCSK_PCA = './mcmc/mcmc_PCSK_PCA/chain.h5'

PATH_h5file_Scikit_noPCA = './mcmc/mcmc_Scikit_noPCA/chain.h5'
PATH_h5file_Scikit_PCA = './mcmc/mcmc_Scikit_PCA/chain.h5'

PCGP_PCA_DATA = read_h5_file(PATH_h5file_PCGP_PCA)
PCGP_noPCA_DATA = read_h5_file(PATH_h5file_PCGP_noPCA)

PCSK_PCA_DATA = read_h5_file(PATH_h5file_PCSK_PCA)
PCSK_noPCA_DATA = read_h5_file(PATH_h5file_PCSK_noPCA)

Scikit_PCA_DATA = read_h5_file(PATH_h5file_Scikit_PCA)
Scikit_noPCA_DATA = read_h5_file(PATH_h5file_Scikit_noPCA)


In [ ]:
plot_chain(PCGP_PCA_DATA[0],PCGP_PCA_DATA[1],PCGP_PCA_DATA[2],PCGP_PCA_DATA[3],'chain_PCGP_PCA')
plot_chain(PCGP_noPCA_DATA[0],PCGP_noPCA_DATA[1],PCGP_noPCA_DATA[2],PCGP_noPCA_DATA[3],'chain_PCGP_noPCA')

plot_chain(PCSK_PCA_DATA[0],PCSK_PCA_DATA[1],PCSK_PCA_DATA[2],PCSK_PCA_DATA[3],'chain_PCSK_PCA')
plot_chain(PCSK_noPCA_DATA[0],PCSK_noPCA_DATA[1],PCSK_noPCA_DATA[2],PCSK_noPCA_DATA[3],'chain_PCSK_noPCA')

plot_chain(Scikit_PCA_DATA[0],Scikit_PCA_DATA[1],Scikit_PCA_DATA[2],Scikit_PCA_DATA[3],'chain_Scikit_PCA')
plot_chain(Scikit_noPCA_DATA[0],Scikit_noPCA_DATA[1],Scikit_noPCA_DATA[2],Scikit_noPCA_DATA[3],'chain_Scikit_noPCA')

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data/example_data_test.pkl')
plot_corner_compare_datasets(PCGP_PCA_DATA[2],PCGP_PCA_DATA[3],PCGP_noPCA_DATA[3],real_parameters,'PCGP_corner')
plot_corner_compare_datasets(PCSK_PCA_DATA[2],PCSK_PCA_DATA[3],PCSK_noPCA_DATA[3],real_parameters,'PCSK_corner')
plot_corner_compare_datasets(Scikit_PCA_DATA[2],Scikit_PCA_DATA[3],Scikit_noPCA_DATA[3],real_parameters,'Scikit_corner')

In [ ]:
real_parameters = read_real_parameters('./separate_training_posterior_data/example_data_test.pkl')
plot_eta_s_posterior(real_parameters,PCGP_PCA_DATA,'eta_s_posterior')
plot_zeta_s_posterior(real_parameters,PCGP_PCA_DATA,'zeta_s_posterior')
plot_yloss_posterior(real_parameters,PCGP_PCA_DATA,'yloss_posterior')